ACARA posee información histórica sobre la venta de autos 0km
en la Argentina. Posee un RDD con información de cada modelo
(marca, modelo, motor, transmisión, origen) y otro con
la información de ventas (marca, modelo, fecha,
concesionario).
Se desea conocer, para los modelos de origen nacional, cuales son
los modelos que ya se discontinuaron (un modelo discontinuado es
aquel que no tuvo ventas en los últimos 12 meses), obteniendo un
nuevo RDD con (marca, modelo, total_vendido,
fecha_inicio_venta, fecha_discontinuación), donde la
fecha de discontinuación es la fecha cuando se vendió el último auto
de ese modelo, ordenado ascendentemente por esta fecha. (***) (
15pts)
Aclaración: Se puede asumir que el primer RDD tiene un único
registro para cada Marca y Modelo.

In [47]:
#modelo (marca, modelo, motor, transmisión, origen) rdd1
data1 = [
    (1,2,2,"a1","ARG"),
    (2,2,2,"a2","ARG"),
    (1,3,2,"a3","ARG"),
    (1,5,2,"a3","ARG"),
    (1,4,2,"a4","Bol")
]
#venta (marca, modelo, fecha, concesionario). rdd2
data2 = [
    (1,2,"2018-08-12", 1),
    (1,3,"2019-08-14", 4),
    (1,4,"2019-07-13", 1),
    (2,2,"2019-08-14", 1),
    (2,2,"2019-08-14", 1),
    (1,3,"2017-07-12", 1),
    (1,3,"2019-08-12", 1),
    (1,3,"2010-08-12", 1),
    (1,2,"2018-08-12", 1),
    (1,5,"2018-08-14", 4),
    (1,5,"2010-08-14", 4),
    (1,4,"2019-07-13", 1),
    (2,2,"2019-08-14", 1),
    (2,2,"2019-08-14", 1),
    (1,3,"2017-07-12", 1),
    (1,3,"2019-08-12", 1),
    (1,3,"2019-08-12", 1),
]

In [48]:
rdd1= sc.parallelize(data1)

rdd2= sc.parallelize(data2)

In [49]:
rdd1 = rdd1.filter(lambda x: x[4]=="ARG")

In [50]:
rdd2 = rdd2.map(lambda x: ((x[0],x[1]),x[2]))
#(marca, modelo ),(fecha)

rdd1 = rdd1.map(lambda x: ((x[0],x[1]), 1))
#(marca, modelo),1

In [51]:
merged = rdd1.join(rdd2)
#(marca, modelo),(fecha,1)

In [52]:
merged = merged.reduceByKey(lambda x,y: (x[0]+y[0],x[1] if x[1]<y[1] else y[1], x[1] if x[1]>y[1] else y[1] ))
#(marca, modelo),(cant total,fecha inicio de venta, ult fecha de venta)

In [53]:
result = merged.filter(lambda x: len(x[1])==3 and x[1][2]>"2018-11-14")

In [54]:
#(marca, modelo, total_vendido, fecha_inicio_venta, fecha_discontinuación),
result = result.map(lambda x: (x[0][0],x[0][1],x[1][0],x[1][1],x[1][2]) )

In [55]:
#El collect solo para verificar
result.collect()

[(1, 3, 7, '2010-08-12', '2019-08-12'), (2, 2, 4, '2019-08-14', '2019-08-14')]